# imports

In [48]:
import pandas as pd
import numpy as np 
import re

pd.set_option('display.max_colwidth', None)

In [51]:
from IPython.display import display, HTML

# Data

In [2]:
df = pd.read_csv('SBAnational.csv')

/var/folders/d0/5dr5q0n145vcn3_m9dxy1xq80000gn/T/ipykernel_40339/2640212651.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('SBAnational.csv')


In [3]:
df1 = df.copy()

# data dictionary

Variable Name	    Description
LoanNr_ChkDgt	    Identifier Primary key
Name	            Borrower name
City	            Borrower city
State	            Borrower state
Zip	                Borrower zip code
Bank	            Bank name
BankState	        Bank state
NAICS	            North American industry classification system code
ApprovalDate	    Date SBA commitment issued
ApprovalFY	        Fiscal year of commitment
Term	            Loan term in months
NoEmp	            Number of business employees
NewExist	        1 = Existing business, 2 = New business
CreateJob	        Number of jobs created
RetainedJob	        Number of jobs retained
FranchiseCode	    Franchise code, (00000 or 00001) = No franchise
UrbanRural	        1 = Urban, 2 = rural, 0 = undefined
RevLineCr	        Revolving line of credit: Y = Yes, N = No
LowDoc	            LowDoc Loan Program: Y = Yes, N = No
ChgOffDate	        The date when a loan is declared to be in default
DisbursementDate	Disbursement date
DisbursementGross	Amount disbursed
BalanceGross	    Gross amount outstanding
MIS_Status	        Loan status charged off = CHGOFF, Paid in full =PIF
ChgOffPrinGr	    Charged-off amount
GrAppv	            Gross amount of loan approved by bank
SBA_Appv	        SBA’s guaranteed amount of approved loan

# prepare

## missing values

In [4]:
df.isna().sum()

LoanNr_ChkDgt             0
Name                     14
City                     30
State                    14
Zip                       0
Bank                   1559
BankState              1566
NAICS                     0
ApprovalDate              0
ApprovalFY                0
Term                      0
NoEmp                     0
NewExist                136
CreateJob                 0
RetainedJob               0
FranchiseCode             0
UrbanRural                0
RevLineCr              4528
LowDoc                 2582
ChgOffDate           736465
DisbursementDate       2368
DisbursementGross         0
BalanceGross              0
MIS_Status             1997
ChgOffPrinGr              0
GrAppv                    0
SBA_Appv                  0
dtype: int64

### name

In [5]:
# there are 14 names that are missing. I'm dropping names, 
# I don't want to give loans based on how catchy or popular the name is.
df.drop(columns= "Name", inplace= True)

#### i want to take the bank, and the state, and find the citys attached to that area. then fill the nulls with that city found.

In [6]:
df[df.City.isna()].head(1)

,LoanNr_ChkDgt,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
42535,1330603010,NaN,UT,84109,MOUNTAIN W. SMALL BUS. FINAN,UT,0,15-Dec-81,1982,300,...,N,N,NaN,6-Jan-82,"$190,000.00",$0.00,P I F,$0.00,"$190,000.00","$190,000.00"


In [7]:
df.dropna(subset= ['City'],inplace= True)

### State

In [8]:
# 14 missing values, ill come back and try the previously mentioned strategy
df.dropna(subset= ['State'],inplace= True)

### Bank

In [9]:
# 1559 missing values, ill come back and try the previously mentioned strategy
df.dropna(subset= ['Bank'],inplace= True)

### BankState

In [10]:
df[df.Bank == 'KOREA EXCHANGE BANK']

,LoanNr_ChkDgt,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
145106,2060334001,NEW YORK,NY,10001,KOREA EXCHANGE BANK,NaN,421940,15-May-98,1998,36,...,0,Y,NaN,31-Jul-98,"$30,000.00",$0.00,P I F,$0.00,"$30,000.00","$24,000.00"


In [11]:
df.BankState = df.BankState.fillna('Foreign')

### NewExist

In [12]:
df[df.NewExist.isna()].head(1)

,LoanNr_ChkDgt,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
395231,3808405000,BETHLEHEM,NH,3574,FIRST COLEBROOK BANK,NH,423310,15-Dec-09,2010,72,...,N,N,NaN,15-Dec-09,"$117,000.00",$0.00,P I F,$0.00,"$117,000.00","$99,450.00"


In [13]:
df.dropna(subset='NewExist', inplace= True)

### RevLineCr  

In [14]:
# im gonna drop these 4500, due to being such a small percentage of data* ill come back and investigate this.
df[df.RevLineCr.isna()].head(1)

,LoanNr_ChkDgt,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
2377,1015874006,OVERLAND PARK,KS,66206,BANK OF BLUE VALLEY,KS,0,6-Mar-97,1997,84,...,NaN,Y,NaN,31-May-97,"$82,000.00",$0.00,P I F,$0.00,"$82,000.00","$65,600.00"


In [15]:
df.dropna(subset='RevLineCr', inplace= True)

### LowDoc

In [16]:
df.dropna(subset='LowDoc', inplace= True)

### MIS_Status

In [62]:
df.dropna(subset='MIS_Status', inplace= True)

### Disbursement Date

In [ ]:
df.dropna(subset='LowDoc', inplace= True)

In [ ]:
DisbursementDate       2368

# Data Types

#### NewExist

In [19]:
df.NewExist = df.NewExist.astype('int64')

#### DisbursementGross

In [ ]:
df.dropna(subset='DisbursementGross', inplace= True)

### DisbursementDate

In [63]:
df.dropna(subset='DisbursementDate', inplace= True)

In [20]:
# define a function to convert a single string value
def convert_price_string(s):
    s = re.sub(r'[^\d\.]', '', s)
    return int(float(s))

In [21]:
# apply the function to the whole column
df['DisbursementGross'] = df['DisbursementGross'].apply(lambda s: convert_price_string(s))


# Balance Gross

In [22]:
# apply the function to the whole column
df['BalanceGross'] = df['BalanceGross'].apply(lambda s: convert_price_string(s))

# Dollar object to Int

In [23]:
# apply the function to the whole column
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(lambda s: convert_price_string(s))

In [24]:
# apply the function to the whole column
df['GrAppv'] = df['GrAppv'].apply(lambda s: convert_price_string(s))

In [25]:
# apply the function to the whole column
df['SBA_Appv'] = df['SBA_Appv'].apply(lambda s: convert_price_string(s))

# data types

In [26]:
# dropping 	LoanNr_ChkDgt due to being a primary key, index is fine.
df.drop(columns= 'LoanNr_ChkDgt',inplace=True)

# date time fixes

In [36]:
# convert the 'dated' columns to datetime format
df['ApprovalDate'] = pd.to_datetime(df['ApprovalDate'], format='%d-%b-%y')

In [37]:
df['DisbursementDate'] = pd.to_datetime(df['DisbursementDate'], format='%d-%b-%y')

In [38]:
df['ChgOffDate'] = pd.to_datetime(df['ChgOffDate'], format='%d-%b-%y')

# date time fixes

In [ ]:
df['DisbursementDate'] = pd.to_datetime(df['DisbursementDate'], format='%d-%b-%y')

In [39]:
nums.dtypes

Zip                           int64
NAICS                         int64
ApprovalDate         datetime64[ns]
Term                          int64
NoEmp                         int64
NewExist                      int64
CreateJob                     int64
RetainedJob                   int64
FranchiseCode                 int64
UrbanRural                    int64
DisbursementDate     datetime64[ns]
DisbursementGross             int64
BalanceGross                  int64
ChgOffPrinGr                  int64
GrAppv                        int64
SBA_Appv                      int64
dtype: object

In [43]:
cats

,City,State,Bank,BankState,ApprovalFY,RevLineCr,LowDoc,ChgOffDate,MIS_Status
0,EVANSVILLE,IN,FIFTH THIRD BANK,OH,1997,N,Y,NaN,P I F
1,NEW PARIS,IN,1ST SOURCE BANK,IN,1997,N,Y,NaN,P I F
2,BLOOMINGTON,IN,GRANT COUNTY STATE BANK,IN,1997,N,N,NaN,P I F
3,BROKEN ARROW,OK,1ST NATL BK & TR CO OF BROKEN,OK,1997,N,Y,NaN,P I F
4,ORLANDO,FL,FLORIDA BUS. DEVEL CORP,FL,1997,N,N,NaN,P I F
...,...,...,...,...,...,...,...,...,...
899159,UPPER ARLINGTON,OH,JPMORGAN CHASE BANK NATL ASSOC,IL,1997,0,N,NaN,P I F
899160,COLUMBUS,OH,JPMORGAN CHASE BANK NATL ASSOC,IL,1997,Y,N,NaN,P I F
899161,SANTA MARIA,CA,"RABOBANK, NATIONAL ASSOCIATION",CA,1997,N,N,NaN,P I F
899162,HONOLULU,HI,BANK OF HAWAII,HI,1997,N,Y,8-Mar-00,CHGOFF


In [29]:
nums = df.select_dtypes(exclude = object)
cats = df.select_dtypes(include = object)

# Overall Exploration DataFrame

In [66]:
df.to_csv('explore.csv')